In [ ]:
import requests
import warnings
import math

import numpy as np
import pandas as pd
import datetime as dt
from datetime import date

import plotly.express as px

from tqdm import tqdm
from gql import gql, Client # to use GraphQL
from gql.transport.requests import RequestsHTTPTransport

warnings.filterwarnings('ignore')

# Кнопка бабло

бла бла бла

## Заполнение датафрэйма

Взял сишку

In [ ]:
df = pd.read_csv('sr_full_date.csv', sep=",")
df = df.sort_values(by=['ticker', 'tradedate', 'tradetime']).drop_duplicates().reset_index().drop('index', axis=1)
df['datetime'] = pd.to_datetime(df['tradedate'] + ' ' + df['tradetime']) 
df = df.drop(['tradedate', 'tradetime'], axis=1) 
df = df[df['clgroup'] == 'YUR'].reset_index().drop('index', axis=1)
df = df[['datetime', 'pos']]
data_pos = df
data_pos

In [ ]:
df = pd.read_csv('SR.csv', sep=",")
df.reset_index(inplace=True)

#df.columns = df.columns[1:].insert(16, 'index')
#df = df.iloc[:, :-1]
df['datetime'] = pd.to_datetime(df['datetime'], dayfirst=True)
df['datetime'] = df['datetime'].dt.strftime('%Y-%m-%d')
df['datetime'] = pd.to_datetime(df['datetime'])
df = df[['datetime', 'weighted_average_price_per_lot']]
df.rename(columns={'weighted_average_price_per_lot': 'price'}, inplace=True)
data_price = df
data_price

In [ ]:
price_dict = dict(zip(data_price['datetime'].dt.date, data_price['price']))

data_pos['price'] = data_pos['datetime'].dt.date.map(price_dict)

all_data = data_pos.sort_values('datetime').reset_index(drop=True)
all_data

## Выбросы

In [ ]:
df = all_data.copy()
df['date'] = df['datetime'].dt.date
#df = df[df['date'] < pd.to_datetime('2021-11-01').date()]
print(df)

fig = px.line(df, x='date', y='price', title='График цен по времени', labels={'date': 'Дата и время', 'price': 'Цена'})
fig.show()

In [ ]:
fig = px.line(df, x='datetime', y='pos', title='График цен по времени', labels={'datetime': 'Дата и время', 'pos': 'Цена'})
fig.update_xaxes(type='category')
fig.show()

In [ ]:
mean_pos = df['pos'].mean()
std_pos = df['pos'].std()
print(mean_pos, std_pos)

outliers_up = pd.DataFrame(columns=['datetime', 'pos', 'price', 'date'])
outliers_down = pd.DataFrame(columns=['datetime', 'pos', 'price', 'date'])
for i in range(len(df['pos'])-1):
    if (df['pos'][i + 1] > (mean_pos + 0.5 * std_pos)) and (df['pos'][i] < (mean_pos + 0.5 * std_pos)):
        outliers_up.loc[len(outliers_up)] = df.iloc[i + 1]
        #outliers_up.append(df.iloc[i + 1], ignore_index=True)
    elif (df['pos'][i + 1] < (mean_pos + 0.5 * std_pos)) and (df['pos'][i] > (mean_pos + 0.5* std_pos)):
        outliers_down.loc[len(outliers_down)] = df.iloc[i + 1]
    elif (df['pos'][i + 1] < (mean_pos - 0.5 * std_pos)) and (df['pos'][i] > (mean_pos - 0.5 * std_pos)):
        outliers_down.loc[len(outliers_down)] = df.iloc[i + 1]
    elif (df['pos'][i + 1] > (mean_pos - 0.5 * std_pos)) and (df['pos'][i] < (mean_pos - 0.5 * std_pos)):
        outliers_up.loc[len(outliers_up)] = df.iloc[i + 1]

print(outliers_up)
print(outliers_down)

In [ ]:
fig = px.line(df, x='datetime', y='price', title='График цен по времени', labels={'datetime': 'Дата и время', 'price': 'Цена'})
fig.add_trace(px.scatter(outliers_up, x='datetime', y='price').update_traces(marker=dict(color='green')).data[0])
fig.add_trace(px.scatter(outliers_down, x='datetime', y='price').update_traces(marker=dict(color='red')).data[0])
fig.update_xaxes(type='category')
fig.show()

In [ ]:
fig = px.line(df, x='datetime', y='pos', title='График цен по времени', labels={'datetime': 'Дата и время', 'pos': 'Pos'})
fig.add_shape(
    dict(
        type="line",
        x0=df['datetime'].min(),
        x1=df['datetime'].max(),
        y0=mean_pos + 2 * std_pos,
        y1=mean_pos + 2 * std_pos,
        line=dict(color="green", width=2)
    )
)
fig.add_shape(
    dict(
        type="line",
        x0=df['datetime'].min(),
        x1=df['datetime'].max(),
        y0=mean_pos - 2 * std_pos,
        y1=mean_pos - 2 * std_pos,
        line=dict(color="red", width=2)
    )
)
fig.update_xaxes(type='category')
fig.show()

In [ ]:
df['pos_diff'] = df['pos'].diff()
df = df.dropna(subset=['pos_diff'])

mean_pos_diff = df['pos_diff'].mean()
std_pos_diff = df['pos_diff'].std()
print(mean_pos_diff, std_pos_diff)

outliers_up_diff = df[df['pos_diff'] > (mean_pos_diff + 20 * std_pos_diff)]
outliers_down_diff = df[df['pos_diff'] < (mean_pos_diff - 20 * std_pos_diff)]
print(outliers_up_diff)
print(outliers_down_diff)

In [ ]:
fig = px.line(df, x='datetime', y='pos_diff', title='График цен по времени', labels={'datetime': 'Дата и время', 'pos_diff': 'Pos_diff'})
fig.add_shape(
    dict(
        type="line",
        x0=df['datetime'].min(),
        x1=df['datetime'].max(),
        y0=mean_pos_diff + 10 * std_pos_diff,
        y1=mean_pos_diff + 10 * std_pos_diff,
        line=dict(color="green", width=2)
    )
)
fig.add_shape(
    dict(
        type="line",
        x0=df['datetime'].min(),
        x1=df['datetime'].max(),
        y0=mean_pos_diff - 10 * std_pos_diff,
        y1=mean_pos_diff - 10 * std_pos_diff,
        line=dict(color="red", width=2)
    )
)
fig.update_xaxes(type='category')
fig.show()

In [ ]:
#Q1 = np.percentile(df['pos_diff'], 25)
#Q3 = np.percentile(df['pos_diff'], 75)
Q1 = df['pos_diff'].quantile(0.25)
Q3 = df['pos_diff'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

anomalies = df[(df['pos_diff'] < lower_bound) | (df['pos_diff'] > upper_bound)]
anomalies

In [ ]:
window_size = 10000  # Пример размера окна
df['pos_ma'] = df['pos'].rolling(window=window_size).mean()
mean_pos_ma = df['pos_ma'].mean()
std_pos_ma = df['pos_ma'].std()
print(mean_pos_ma, std_pos_ma)
fig = px.line(df, x='datetime', y='pos_ma', title='График pos MA10000 по времени', labels={'datetime': 'Дата и время', 'pos_ma': 'Pos MA10'})
fig.add_shape(
    dict(
        type="line",
        x0=df['datetime'].min(),
        x1=df['datetime'].max(),
        y0=mean_pos_ma + 1 * std_pos_ma,
        y1=mean_pos_ma + 1 * std_pos_ma,
        line=dict(color="green", width=2)
    )
)
fig.add_shape(
    dict(
        type="line",
        x0=df['datetime'].min(),
        x1=df['datetime'].max(),
        y0=mean_pos_ma - 1 * std_pos_ma,
        y1=mean_pos_ma - 1 * std_pos_ma,
        line=dict(color="red", width=2)
    )
)
fig.update_xaxes(type='category')
fig.show()